In [20]:
# carregando bibliotecas
import pandas as pd


In [22]:
# Função que carrega dataset e prepara arquivo para análise em texto.
def convert_table_to_gpt_format(file_path, peps_list=None):
    """
    Lê um arquivo Excel e aplica padrões de detecção AML, além de converter linhas em strings descritivas.

    :param file_path: Caminho para o arquivo Excel.
    :param peps_list: Lista opcional de PEPs (Pessoas Expostas Politicamente).
    :return: lista de strings descritivas e alertas detectados
    """
    # Carregar o arquivo Excel
    dataframe = pd.read_excel(file_path)

    # Lista para armazenar descrições e alertas
    descriptions = []
    alerts = []

    for _, row in dataframe.iterrows():
        description = (
            f"Transação ID {row['transaction_id']}: Em {row['timestamp']}, "
            f"foi realizada uma transação no valor de R$ {row['amount']:.2f}. "
            f"A parte debitada foi identificada como {row['debit_party']} "
            f"e a parte creditada como {row['credit_party']}. "
            f"O status da transação é '{row['status']}' e as notas adicionais indicam: {row['notes']}."
        )
        descriptions.append(description)

        # Detectar transações acima de R$ 100.000,00
        if row['amount'] > 100000.00:
            alerts.append(f"ALERTA: Transação {row['transaction_id']} excede R$ 100.000,00.")

        # Detectar envolvimento de PEPs
        if peps_list and (row['debit_party'] in peps_list or row['credit_party'] in peps_list):
            alerts.append(f"ALERTA: Transação {row['transaction_id']} envolve uma PEP ({row['debit_party']} ou {row['credit_party']}).")

    # Detectar transferências em alta frequência
    dataframe['timestamp'] = pd.to_datetime(dataframe['timestamp'])
    dataframe.sort_values(by=['debit_party', 'timestamp'], inplace=True)
    dataframe['time_diff'] = dataframe.groupby('debit_party')['timestamp'].diff().dt.total_seconds()

    for _, row in dataframe.iterrows():
        if pd.notnull(row['time_diff']) and row['time_diff'] < 3600:  # Intervalo menor que 1 hora
            alerts.append(f"ALERTA: Alta frequência detectada para {row['debit_party']} na transação {row['transaction_id']}.")

    return descriptions, alerts

# Exemplo de uso
file_path = 'fictitious_pix_data (1).xlsx'
peps_list = ['CPF59095', 'CNPJ32900']  # Exemplo de lista de PEPs
descriptions, alerts = convert_table_to_gpt_format(file_path, peps_list=peps_list)

# Exibir os primeiros exemplos para validação
print(descriptions[:5])
print(alerts[:5])


["Transação ID 1: Em 2024-12-15 01:16:00, foi realizada uma transação no valor de R$ 461981.83. A parte debitada foi identificada como CPF59095 e a parte creditada como CNPJ28807. O status da transação é 'pending' e as notas adicionais indicam: Transferência PIX.", "Transação ID 2: Em 2024-12-15 15:35:00, foi realizada uma transação no valor de R$ 323451.82. A parte debitada foi identificada como CPF29101 e a parte creditada como CNPJ32900. O status da transação é 'pending' e as notas adicionais indicam: Transferência comum.", "Transação ID 3: Em 2024-12-15 00:56:00, foi realizada uma transação no valor de R$ 142069.57. A parte debitada foi identificada como CPF48141 e a parte creditada como CNPJ17002. O status da transação é 'blocked' e as notas adicionais indicam: Bloqueio automático.", "Transação ID 4: Em 2024-12-15 15:11:00, foi realizada uma transação no valor de R$ 66370.58. A parte debitada foi identificada como CPF89208 e a parte creditada como CNPJ22299. O status da transação 

In [ ]:
# criando função

def convert_table_to_gpt_format(dataframe):
    """
    Converte cada linha de um DataFrame em uma string descritiva para leitura pelo GPT.

    :param dataframe: pandas.DataFrame
    :return: lista de strings descritivas
    """
    descriptions = []

    for _, row in dataframe.iterrows():
        description = (
            f"Transação ID {row['transaction_id']}: Em {row['timestamp']}, "
            f"foi realizada uma transação no valor de R$ {row['amount']:.2f}. "
            f"A parte debitada foi identificada como {row['debit_party']} "
            f"e a parte creditada como {row['credit_party']}. "
            f"O status da transação é '{row['status']}' e as notas adicionais indicam: {row['notes']}."
        )
        descriptions.append(description)

    return descriptions

# Testando a função com a tabela carregada
gpt_ready_data = convert_table_to_gpt_format(df)

# Exibir os primeiros exemplos para validação
print(gpt_ready_data[:5])
